# Dyadic Δ Scan Notebook  
This Colab‑ready notebook reproduces and **stress‑tests** the identity  

\[
\Delta_C(n)=\min_i \bigl|\,C-(i/2^n)^2\bigr|\;\approx\;\kappa_C\,2^{-2n}
\]

for a configurable list of physical & mathematical constants over an arbitrary bit‑depth `n_max`.

**Sections**

1. **Setup** – install / import.  
2. **Parameter block** – pick constants & `n_max`.  
3. **Sweep function** – compute best Δ and residue class.  
4. **Run sweep** – generates a Pandas dataframe.  
5. **Plots** – log‑log slope + phase histogram.  
6. **Regression & κ‑plateau** – numeric diagnostics.  
7. **Sandbox** – cell for your own experiments.

> *Edit the parameter block only; everything else will update automatically.*  


In [1]:
# Install dependencies (lightweight; skip if already present)
!pip -q install mpmath pandas matplotlib seaborn tqdm scipy --upgrade

import mpmath as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from scipy import stats

mp.mp.dps = 500   # precision (~1700 bits)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
plotnine 0.14.6 requires scipy<1.16.0,>=1.8.0, but you have scipy 1.16.0 which is incompatible.


In [2]:
# ==== Parameters ==============================================================
n_max = 200  # bits of precision to sweep

CONSTANTS = {
    "alpha"    : 1/137.035999,                              # fine‑structure
    "phi"        : (1+5**0.5)/2,                              # golden ratio
    "pi"         : mp.pi,
    "pi/e"       : mp.pi/mp.e,
    "sqrt3/pi"   : (3**0.5)/mp.pi,
    "sqrt2"      : mp.sqrt(2),
    "e"          : mp.e,
    "e/pi tower" : 0.880367778981734621826749
    "pi/e tower" : 1.187523635359249905438407
    "lil phi tower" : 0.710439287156503188669345
    "1/2 tower" : 0.641185744504985984486200
}

# ==============================================================================


SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-2-303239698.py, line 12)

In [ ]:
def best_delta(C, n):
    """Return (delta, numerator) for best dyadic approximation of sqrt(C) at bit‑depth n."""
    scale = mp.mpf(1) << n                # 2**n as mp.mpf
    est   = int(mp.nint(mp.sqrt(C)*scale))
    best  = mp.mpf('inf'); best_i = None
    for i in range(est-3, est+4):         # small local search
        d = abs((i/scale)**2 - C)
        if d < best:
            best, best_i = d, i
    return best, best_i

def sweep_constants(constants, n_max):
    rows = []
    for name, C in constants.items():
        for n in tqdm(range(1, n_max+1), desc=f"{name:8s}"):
            delta, i_star = best_delta(C, n)
            rows.append({
                "constant" : name,
                "C"        : C,
                "n"        : n,
                "delta"    : delta,
                "log10_delta": mp.log10(delta),
                "i_star"   : i_star,
                "phase"    : i_star % 4,
                "kappa_n"  : (2**(2*n))*delta,    # κ estimate
            })
    return pd.DataFrame(rows)

df = sweep_constants(CONSTANTS, n_max)
df.to_pickle("delta_scan.pkl")
print("Saved dataframe -> delta_scan.pkl")


In [ ]:
# --- Log‑log slope plot -------------------------------------------------------
plt.figure(figsize=(8,5))
for name, grp in df.groupby('constant'):
    plt.plot(grp['n'], grp['log10_delta'], label=name)
plt.xlabel("n (bits)")
plt.ylabel("log10 Δ")
plt.title("Δ decay across constants")
plt.legend()
plt.tight_layout()
plt.show()

# --- Phase histogram ----------------------------------------------------------
plt.figure(figsize=(6,4))
sns.countplot(x='phase', hue='constant', data=df)
plt.title("Residue class of best numerator (i_* mod 4)")
plt.tight_layout()
plt.show()


In [ ]:
print("\n=== Slope diagnostics (OLS on log₂ Δ vs n) ===")
for name, grp in df.groupby('constant'):
    x = grp['n'].astype(float)
    y = np.log2(grp['delta'].astype(float))
    slope, intercept, r, p, se = stats.linregress(x, y)
    print(f"{name:8s}  slope = {slope:.5f}  R² = {r**2:.5f}  p = {p:.2e}")

print("\n=== κ plateau (median of last 20 n) ===")
for name, grp in df.groupby('constant'):
    plateau = grp[grp['n'] > (n_max-20)]['kappa_n'].median()
    print(f"{name:8s}  κ ≈ {plateau:.3e}")



## Sandbox  
Use the empty cell below to try different constants, plot individual κ‑trajectories, or extend to cubes / other bases.


In [ ]:
# Example: plot κ_n for a single constant
const = 'alpha'
sub   = df[df['constant']==const]
plt.plot(sub['n'], sub['kappa_n'])
plt.title(f"κ_n trajectory for {const}")
plt.xlabel('n'); plt.ylabel('κ_n')
plt.show()
